# ファイルの入出力

Polarsでは、大規模なデータセットを効率的に扱うための高速なファイル入出力操作が提供されています。データを読み込んだり書き出したりする際に、さまざまなフォーマットに対応しており、迅速なデータ処理をサポートします。この章では、Polarsを使用したファイルの入出力操作方法について詳しく説明します。

In [29]:
import polars as pl
from helper.jupyter import row, capture_except

## CSVファイル

CSVファイルを読み込む際には、ファイル構造やデータの特性に応じて柔軟に操作する必要があります。本セクションでは、Polarsを使用してさまざまなCSVファイルを読み込む方法を紹介します。

### ヘッダー

CSVファイルには、ヘッダーの有無や、ヘッダーが複数行にわたる場合があります。以下のデータを例に、ヘッダーの扱い方について説明します。

In [ ]:
%%writefile data/csv_header.csv
A,B
a,b
0,1
2,3
4,5

- `df1`: デフォルト設定では、CSVファイルをヘッダー付きとして読み込みます。この場合、データの先頭行が列の名前として解釈されます。
- `df2`: `has_header=False`を指定することで、CSVの先頭行をデータとして扱います。この場合、`new_columns`引数を使用して列名を自分で指定できます。
- `df3`: `skip_rows`引数を指定することで、最初のN行をスキップしてからデータを読み込むことができます。
- `df4`: `skip_rows_after_header`引数を指定することで、ヘッダー行の次のN行をスキップしてデータを読み込みます。
- `df5`: 最初の2行をヘッダーなしで読み込んで、それぞれの列を結合した結果を`new_columns`引数に渡し、新しい列名として適用します。この方法を使うことで、複数行のヘッダーを柔軟に扱うことができます。

これらの方法を活用することで、CSVデータの構造に応じた柔軟な読み込みが可能になります。

In [30]:
fn = 'data/csv_header.csv'
df1 = pl.read_csv(fn)
df2 = pl.read_csv(fn, has_header=False, new_columns=['x', 'y'])
df3 = pl.read_csv(fn, skip_rows=1)
df4 = pl.read_csv(fn, skip_rows_after_header=1)

df_header = pl.read_csv(fn, n_rows=2, has_header=False)
columns = df_header.select(pl.all().str.join('-')).row(0)
df5 = pl.read_csv(fn, has_header=False, skip_rows=2, new_columns=columns)
row(df1, df2, df3, df4, df5)

A,B
str,str
"""a""","""b"""
"""0""","""1"""
"""2""","""3"""
"""4""","""5"""
x,y
str,str
"""A""","""B"""
"""a""","""b"""
"""0""","""1"""


### 列のデータ型

`infer_schema`引数がデフォルト値`True`の場合、`infer_schema_length`引数で指定された先頭の行数を使用して各列のデータ型を推定します。この範囲を超えて異なるデータ型の値が出現した場合、エラーが発生します。以下のデータを例に、データ型の扱い方について説明します。

In [ ]:
%%writefile data/csv_different_type.csv
A,B
0,1
2,3
4,5
a,5.5
10,20

`infer_schema_length`のデフォルト値は100ですが、以下のコードでは、`infer_schema_length`を2行に設定してエラーを発生させます。

In [32]:
%%capture_except
df = pl.read_csv('data/csv_different_type.csv', infer_schema_length=2)

ComputeError: could not parse `a` as dtype `i64` at column 'A' (column number 1)

The current offset in the file is 15 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `schema_overrides` argument
- setting `ignore_errors` to `True`,
- adding `a` to the `null_values` list.

Original error: ```remaining bytes non-empty```


エラーメッセージにはいくつかの解決方法が示されています。以下はそれらの方法を使用してデータを読み込む例です。

- **`df1`**: `infer_schema_length`引数で推定行数を増やすことで、A列のデータ型を`str`、B列を`f64`として読み込みます。

- **`df2`**: `infer_schema_length=None`を指定すると、すべての行を使用してデータ型を推定します。また、`null_values`引数を使用して特定の値をnullと見なすことで、A列を`i64`として読み込みます。

- **`df3`**: `ignore_errors=True`を指定すると、推定データ型に一致しない値をnullとして読み込みます。この場合、A列とB列はどちらも`i64`になります。

- **`df4`**: `schema_overrides`引数を使用して、各列のデータ型を明示的に指定します。さらに、`ignore_errors=True`を指定して不正な値を除外します。`schema_overrides`を使用すると、効率的なデータ型を選択でき、メモリ使用量を削減できます。

これらの方法を使用することで、データ型の推定やエラー処理に柔軟に対応できます。

In [33]:
fn = 'data/csv_different_type.csv'
df1 = pl.read_csv(fn, infer_schema_length=1000)
df2 = pl.read_csv(fn, infer_schema_length=None, null_values=['a'])
df3 = pl.read_csv(fn, infer_schema_length=2, ignore_errors=True)
df4 = pl.read_csv(fn, schema_overrides={'A':pl.Int16, 'B':pl.Float32}, ignore_errors=True)
row(df1, df2, df3, df4)

A,B
str,f64
"""0""",1.0
"""2""",3.0
"""4""",5.0
"""a""",5.5
"""10""",20.0
A,B
i64,f64
0,1.0
2,3.0


### スペース処理

CSVデータ内の列値に末尾のスペースが含まれている場合、Polarsの標準CSVエンジンはこれをそのまま取り込み、列データ型を`str`として解釈します。例えば、次のようなCSVデータを読み込む場合を考えます：

In [ ]:
%%writefile data/csv_trailing_space.csv
str,int,float
abc ,4 ,5.67 
def ,5 ,1.23 

このデータを読み込むと、Polarsの標準エンジンと`use_pyarrow=True`を指定した場合で動作が異なります：

* `df1`: Polarsの標準エンジンでは、すべての列が文字列(`str`)として扱われます。
* `df2`: `use_pyarrow=True`を指定すると、数値列(`int`, `float`)が適切に解釈されます。

In [35]:
fn = 'data/csv_trailing_space.csv'
df1 = pl.read_csv(fn)
df2 = pl.read_csv(fn, use_pyarrow=True)
row(df1, df2)

str,int,float
str,str,str
"""abc ""","""4 ""","""5.67 """
"""def ""","""5 ""","""1.23 """
str,int,float
str,i64,f64
"""abc """,4,5.67
"""def """,5,1.23


Polarsでは文字列列を自動的に数値型に変換するカスタム関数を作成することで、スペースを取り除きつつ適切にキャストできます。以下はその例です。

1. `s.str.strip_chars()` を使用して余分なスペースを削除。
2. `.cast(int_type)` を試みて、整数型に変換できるかを確認。
3. 整数型への変換が失敗した場合は `.cast(float_type)` を試みて、浮動小数型に変換。
4. どちらのキャストも失敗した場合には元の文字列型を返す。

In [37]:
from polars import selectors as cs
from polars.exceptions import InvalidOperationError

# この関数はhelper/polars.pyにあります。
def try_cast_to_number(s, int_type=pl.Int64, float_type=pl.Float64):
    try:
        return s.str.strip_chars().cast(int_type)
    except InvalidOperationError:
        try:
            return s.str.strip_chars().cast(float_type)
        except InvalidOperationError:
            return s

df1.with_columns(cs.string().map_batches(try_cast_to_number))

str,int,float
str,i64,f64
"""abc """,4,5.67
"""def """,5,1.23
